# S05 — Harmonía e análise básica con music21
**Curso:** Programación Musical con Python — 2º Trimestre  
**Duración:** 1 sesión (≈ 1 hora)

## 🎯 Obxectivos
- Crear e manipular **acordes** con `music21`.
- Entender a relación entre **tonalidade** e **graos**.
- Construír progresións típicas (I–V–vi–IV, ii–V–I…).
- Xeración dun **acompañamento automático** moi simple para unha melodía.
- Ler código como partitura: sección de melodía + sección de harmonía.

> 🎹 Hoxe pasamos de “unha liña” a “harmonía”.


## 0) Setup (Colab)


In [ ]:
!pip -q install music21
from music21 import stream, note, chord, meter, tempo, key, roman

print("✅ music21 listo")


## 1) Crear acordes

Un acorde é un conxunto de notas simultáneas.
Exemplos:
- Do maior: C–E–G
- Sol maior: G–B–D


In [ ]:
c_do = chord.Chord(["C4", "E4", "G4"])
c_sol = chord.Chord(["G3", "B3", "D4"])

print("Do:", c_do.commonName, " | notas:", [p.nameWithOctave for p in c_do.pitches])
print("Sol:", c_sol.commonName, " | notas:", [p.nameWithOctave for p in c_sol.pitches])


### Exercicio rápido
Crea tres acordes máis: Am (A–C–E), F (F–A–C), Dm (D–F–A)


In [ ]:
c_am = chord.Chord(["A3","C4","E4"])
c_f  = chord.Chord(["F3","A3","C4"])
c_dm = chord.Chord(["D3","F3","A3"])

[c.commonName for c in [c_am, c_f, c_dm]]


## 2) Tonalidade e graos (cifrado romano)

Nunha tonalidade (p.ex. Do maior) cada nota/ acorde ten unha función:
- I (tónica), V (dominante), IV (subdominante), etc.

`music21` pode crear acordes por cifrado romano.


In [ ]:
k = key.Key("C")
prog = ["I", "V", "vi", "IV"]

acordes = [roman.RomanNumeral(rn, k).pitches for rn in prog]
acordes


Imos converter eses acordes a obxectos `Chord` para tocar/visualizar.


In [ ]:
def acorde_por_rn(rn, tonalidade="C", oitava=3):
    k = key.Key(tonalidade)
    rn_obj = roman.RomanNumeral(rn, k)
    # baixamos a rexión para acompañamento
    pitches = []
    for p in rn_obj.pitches:
        pitches.append(f"{p.name}{oitava}")
    return chord.Chord(pitches)

for rn in ["I","V","vi","IV"]:
    ac = acorde_por_rn(rn, tonalidade="C", oitava=3)
    print(rn, "->", [p.nameWithOctave for p in ac.pitches])


## 3) Construír unha progresión como Stream (harmonía)

Imos crear unha progresión de 4 compases en 4/4:
- cada acorde dura 1 compás (4 pulsos)


In [ ]:
def base_stream(time_sig="4/4", bpm=90, tonalidade="C"):
    s = stream.Stream()
    s.append(meter.TimeSignature(time_sig))
    s.append(tempo.MetronomeMark(number=bpm))
    s.append(key.Key(tonalidade))
    return s

harm = base_stream(bpm=90, tonalidade="C")

progresion = ["I","V","vi","IV"]
for rn in progresion:
    ac = acorde_por_rn(rn, tonalidade="C", oitava=3)
    ac.quarterLength = 4
    harm.append(ac)

harm.show("text")


## 4) Melodía + acompañamento (moi simple)

Facemos dúas liñas:
- melodía (notas individuais)
- acordes (acompañamento)

Usaremos `stream.Part` para separar voces.


In [ ]:
from music21 import stream as m21stream

melodia = ["E4","D4","C4","D4", "E4","E4","E4","REST"]
ritmo   = [1,1,2,  1,1,2,1,1]  # suma 10 (exemplo); imos axustar a 8 pulsos (2 compases)

# Axuste a 2 compases (8 pulsos):
melodia = ["E4","D4","C4","D4","E4","E4","REST","D4"]
ritmo   = [1,1,1,1, 1,1,1,1]

def facer_melodia(alturas, ritmos):
    p = m21stream.Part()
    for a, d in zip(alturas, ritmos):
        if a == "REST":
            p.append(note.Rest(quarterLength=d))
        else:
            p.append(note.Note(a, quarterLength=d))
    return p

def facer_acordes(progresion, compases_por_acorde=1, tonalidade="C"):
    p = m21stream.Part()
    for rn in progresion:
        ac = acorde_por_rn(rn, tonalidade=tonalidade, oitava=3)
        ac.quarterLength = 4 * compases_por_acorde
        p.append(ac)
    return p

part_mel = facer_melodia(melodia, ritmo)

# para 2 compases: 2 acordes (cada un 1 compás)
part_harm = facer_acordes(["I","V"], tonalidade="C")

score = m21stream.Score()
score.append(meter.TimeSignature("4/4"))
score.append(tempo.MetronomeMark(number=90))
score.append(key.Key("C"))

score.insert(0, part_mel)
score.insert(0, part_harm)

score.show("text")


### Exportar (MIDI / MusicXML)


In [ ]:
midi_path = score.write("midi")
xml_path = score.write("musicxml")
midi_path, xml_path


## 5) Exercicio guiado: harmonizar unha melodía (regra simple)

Regra didáctica (suficiente para comezar):
- Se a nota da melodía pertence ao acorde, ese acorde serve.
- Se non, probamos outro acorde próximo.

Aquí farémolo manual: ti decides unha progresión de 4 acordes
e aplicas 1 acorde por compás.

### Pasos
1) Escribe unha melodía de 4 compases (16 pulsos)
2) Escolle 4 acordes (I, V, vi, IV por exemplo)
3) Xera Score con dúas partes e exporta


In [ ]:
# ✅ Espazo de traballo (modifica!)
mel = ["C4","D4","E4","G4","E4","D4","C4","REST",
       "E4","F4","G4","A4","G4","F4","E4","D4"]
rit = [1]*16

part_mel2 = facer_melodia(mel, rit)
part_harm2 = facer_acordes(["I","V","vi","IV"], tonalidade="C")

score2 = m21stream.Score()
score2.append(meter.TimeSignature("4/4"))
score2.append(tempo.MetronomeMark(number=100))
score2.append(key.Key("C"))

score2.insert(0, part_mel2)
score2.insert(0, part_harm2)

score2.show("text")

# Exporta
# score2.write("midi")
# score2.write("musicxml")


## 🧩 Tarefa avaliable (entrega)

### Parte A (código)
- Harmoniza unha melodía de **8 compases** en 4/4.
- Usa unha progresión de **4 acordes** repetida dúas veces (ex: I–V–vi–IV).
- Requisitos:
  - 2 partes: melodía + harmonía
  - polo menos 1 silencio na melodía
  - exportar a MIDI e/ou MusicXML

### Parte B (reflexión: 5 liñas)
- Por que escolliches esa progresión?
- En que compás sentes “tensión” e onde “repouso”?
- Onde se ve a estrutura no código?


## 🧠 Ler código como partitura (checkpoint)
Antes de entregar:
- etiqueta con comentarios: `# MELODÍA`, `# HARMONÍA`, `# PROGRESIÓN`
- marca os compases (cada 4 notas se usas negras)
O obxectivo é que a peza se “lea” só mirando o notebook.
